### Import thư viện

In [ ]:
import numpy as np
import pandas as pd
import requests
import datetime

### Thu thập dữ liệu

- Tây Bắc Bộ: Lào Cai, Yên Bái, Điện Biên, Hòa Bình, Lai Châu, Sơn La

- Đông Bắc Bộ: Hà Giang, Cao Bằng, Bắc Kạn, Lạng Sơn, Tuyên Quang, Thái Nguyên, Phú Thọ, Bắc Giang, Quảng Ninh.

- Đồng bằng sông Hồng: Bắc Ninh, Hà Nam, Hà Nội, Hải Dương, Hưng Yên, Hải Phòng, Nam Định, Ninh Bình, Thái Bình, Vĩnh Phúc



In [ ]:
base_url = 'http://api.weatherapi.com/v1/current.json?'
api_key = 'ac7ffe17e86f41469c950044233006'
city_list = ['Lao Cai', 'Yen Bai', 'Dien Bien', 'Lai Chau', 'Son La', 'Hoa Binh',
             'Ha Giang', 'Cao Bang', 'Lang Son', 'Tuyen Quang', 'Bac Kan', 'Thai Nguyen', 'Yen Bai', 'Phu Tho', 'Bac Giang', 'Quang Ninh'
             'Ha Noi', 'Hai Phong', 'Hung Yen', 'Nam Dinh', 'Thai Binh', 'Hai Duong', 'Hoa Binh', 'Ninh Binh', 'Vinh Phuc', 'Bac Ninh', 'Ha Nam']

In [ ]:
def find_position(city_list, city):
    try:
        position = city_list.index(city)
        return position
    except ValueError:
        print(f"City '{city}' not found in the list")
        return -1

In [ ]:
for item in city_list:
    city = item
    maxtemp = []
    mintemp = []
    avgtemp = []
    maxwind_kph = []
    totalprecip_mm = []
    avgvis_km = []
    avghumidity = []
    comment = []
    UV_index = []
    date = []
    time = "29-06-2022"
    time_dt = datetime.datetime.strptime(time, "%d-%m-%Y").date()
    for i in range(367):
        time_str = time_dt.isoformat()
        api_url = f f'http://api.weatherapi.com/v1/history.json?key={api_key}&q={city}&dt={time_str}'
        response = requests.get(api_url)
        data = response.json()
        dataday = data['forecast']['forecastday'][0]
        day = dataday['day']
        date.append(time_dt)
        maxtemp.append(day['maxtemp_c'])
        mintemp.append(day['mintemp_c'])
        avgtemp.append(day['avgtemp_c'])
        maxwind_kph.append(day['maxwind_kph'])
        totalprecip_mm.append(day['totalprecip_mm'])
        avghumidity.append(day['avghumidity'])
        comment.append(day['condition']['text'])
        UV_index.append(day['uv'])
        time_dt += datetime.timedelta(days=1)
    summary = np.array([date, maxtemp, mintemp, avgtemp, maxwind_kph, totalprecip_mm, avghumidity, comment, UV_index])
    column_values = list(range(1, 368))
    index_values = ['Day','Temperature Max','Temperature Min','Temperature Avg','MaxWind_km/h','TotalPrecip_mm','Humidity Avg','Comment', 'UV Index']
    df = pd.DataFrame(data = summary,
                     index = index_values,
                     columns = column_values)
    position = find_position(city_list, item)
    if position != -1:
        filename = f"{city_list[position]}.csv"
        df.T.to_csv(filename, index=False)